# The Boston Housing Dataset

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per \$10,000
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in $1000's

In [1]:
# This command upgrades the 'numexpr' library using pip
!pip install --upgrade numexpr


# Step 1: Import Necessary Libraries

In [2]:
# Import necessary libraries and modules from SageMaker and boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.inputs import TrainingInput
import boto3
import os


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Step 2: Set Up SageMaker Session and Role

In [3]:
# Create a SageMaker session, which manages interactions with SageMaker services
sagemaker_session = sagemaker.Session()

# Get the IAM execution role used for SageMaker to access AWS resources
role = get_execution_role()

# Get the AWS region associated with the SageMaker session
region = sagemaker_session.boto_region_name


# Step 3: Prepare Your Data

In [4]:
# Import the pandas library as 'pd' for data manipulation and analysis
import pandas as pd

# Import the train_test_split function from scikit-learn for data splitting
from sklearn.model_selection import train_test_split


In [5]:
# Read data from the 'HousingData.csv' file into a pandas DataFrame called 'housing_data'
housing_data = pd.read_csv('HousingData.csv')


In [6]:
# Check for missing values (NaN) in the 'housing_data' DataFrame
housing_data.isna()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,False,False,False,False,False,False,False,False,False,False,False,False,True,False
502,False,False,False,False,False,False,False,False,False,False,False,False,False,False
503,False,False,False,False,False,False,False,False,False,False,False,False,False,False
504,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
# Remove rows with missing values (NaN) from the 'housing_data' DataFrame
housing_data = housing_data.dropna()


In [8]:
# The target variable column is named 'MEDV'

# Create the feature matrix 'X' by dropping the 'MEDV' column
X = housing_data.drop('MEDV', axis=1)

# Create the target variable 'y' by converting 'MEDV' to integers
y = housing_data['MEDV'].astype('int')

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Concatenate the features and labels back into one DataFrame for training data
housing_train_data = pd.concat([y_train, X_train], axis=1)

# Concatenate the features and labels back into one DataFrame for validation data
housing_validation_data = pd.concat([y_val, X_val], axis=1)

# Save the training data to a CSV file without headers and indices
housing_train_data.to_csv('HousingData_train.csv', header=False, index=False)

# Save the validation data to a CSV file without headers and indices
housing_validation_data.to_csv('HousingData_validation.csv', header=False, index=False)


In [10]:
# Concatenate the features and labels back into one DataFrame for training data
housing_train_data = pd.concat([y_train, X_train], axis=1)

# Concatenate the features and labels back into one DataFrame for validation data
housing_validation_data = pd.concat([y_val, X_val], axis=1)

# Save the training data to a CSV file without headers and indices
housing_train_data.to_csv('HousingData_train.csv', header=False, index=False)

# Save the validation data to a CSV file without headers and indices
housing_validation_data.to_csv('HousingData_validation.csv', header=False, index=False)


In [11]:
# Define your Amazon S3 bucket and prefix for data storage
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/boston-housing/classification'

# Paths to your local data files - replace with your actual file paths
local_train = 'HousingData_train.csv'
local_validation = 'HousingData_validation.csv'

# Upload the local training data to the specified S3 bucket and prefix
train_uri = sagemaker_session.upload_data(local_train, bucket=bucket, key_prefix=prefix)

# Upload the local validation data to the specified S3 bucket and prefix
validation_uri = sagemaker_session.upload_data(local_validation, bucket=bucket, key_prefix=prefix)


In [12]:
# Print the S3 URI for the training data
print("Training URI: ", train_uri)

# Print the S3 URI for the validation data
print("Validation URI: ", validation_uri)


Training URI:  s3://sagemaker-us-east-1-256451472834/sagemaker/boston-housing/classification/HousingData_train.csv
Validation URI:  s3://sagemaker-us-east-1-256451472834/sagemaker/boston-housing/classification/HousingData_validation.csv


# Step 4: Get the Linear Learner Image URI

In [13]:
from sagemaker import image_uris

# Retrieve the container image URI for the SageMaker Linear Learner algorithm
container = image_uris.retrieve(framework='linear-learner', region=region)


# Step 5: Configure the SageMaker Linear Learner Estimator

In [14]:
# Calculate the number of rows and features in the 'housing_data' DataFrame
num_rows, num_features = housing_data.shape

# Print the number of rows and features
print("Number of Rows:", num_rows)
print("Number of Features:", num_features)

Number of Rows: 394
Number of Features: 14


In [18]:
linear_learner = sagemaker.estimator.Estimator(container,
# Create a SageMaker Linear Learner estimator
linear_learner = sagemaker.estimator.Estimator(container,
                                               role, 
                                               instance_count=1, 
                                               instance_type='ml.m4.xlarge',
                                               output_path=f's3://{bucket}/{prefix}/output',
                                               sagemaker_session=sagemaker_session)

# Set hyperparameters for the Linear Learner
linear_learner.set_hyperparameters(feature_dim=13,  # Number of input features (excluding target)
                                   mini_batch_size=32,  # Size of mini-batches for training
                                   predictor_type='regressor',  # Specify 'regressor' for regression
                                   normalize_data=True,  # Normalize input features
                                   normalize_label=True)  # Normalize target variable for regression


# Step 6: Train the Model

In [19]:
# Fit the SageMaker Linear Learner estimator to the training and validation data
linear_learner.fit({'train': TrainingInput(train_uri, content_type='text/csv'),
                    'validation': TrainingInput(validation_uri, content_type='text/csv')})

INFO:sagemaker:Creating training-job with name: linear-learner-2024-01-31-13-02-22-394


2024-01-31 13:02:22 Starting - Starting the training job......
2024-01-31 13:02:56 Starting - Preparing the instances for training.........
2024-01-31 13:04:38 Downloading - Downloading input data...
2024-01-31 13:05:08 Downloading - Downloading the training image......
2024-01-31 13:06:19 Training - Training image download completed. Training in progress....
2024-01-31 13:06:54 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[01/31/2024 13:06:43 INFO 140531743000384] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0

# Step 7: Deploy the Endpoint

## Check to see if the Endpoint exixsts and if it does delete it

In [22]:
import boto3
from botocore.exceptions import ClientError

# Create the SageMaker client to interact with the SageMaker service
sagemaker_client = boto3.client('sagemaker')

def check_resource_exists(function, **kwargs):
    """
    Checks if a specified AWS resource exists by calling a provided AWS service
    function with keyword arguments.

    :param function: AWS service function to call (e.g., describe_endpoint)
    :param kwargs: Keyword arguments to pass to the AWS service function
    :return: True if the resource exists, False if not
    """
    try:
        function(**kwargs)  # Attempt to describe the resource
        return True  # If successful, the resource exists
    except ClientError as error:
        # Check if the error is because the resource does not exist
        if error.response['Error']['Code'] in ['ValidationException', 'ResourceNotFoundException']:
            return False  # Resource does not exist
        else:
            raise  # Raise other exceptions as they might indicate different issues

def delete_resource_if_exists(check_function, delete_function, **kwargs):
    """
    Deletes an AWS resource if it exists.

    :param check_function: Function to check if the resource exists
    :param delete_function: Function to delete the resource
    :param kwargs: Keyword arguments for both checking and deleting functions
    """
    if check_resource_exists(check_function, **kwargs):
        resource_name = list(kwargs.values())[0]
        print(f"Deleting existing resource: {resource_name}")
        delete_function(**kwargs)  # Delete the resource
        print(f"Resource {resource_name} deleted.")
    else:
        print(f"Resource {list(kwargs.values())[0]} does not exist or was already deleted.")

# Example usage to delete an endpoint configuration
delete_resource_if_exists(
    check_function=sagemaker_client.describe_endpoint_config,
    delete_function=sagemaker_client.delete_endpoint_config,
    EndpointConfigName="regression-linear-learner-endpoint"
)

# Example usage to delete an endpoint
delete_resource_if_exists(
    check_function=sagemaker_client.describe_endpoint,
    delete_function=sagemaker_client.delete_endpoint,
    EndpointName="regression-linear-learner-endpoint"
)


Deleting existing resource: regression-linear-learner-endpoint
Resource regression-linear-learner-endpoint deleted.
Deleting existing resource: regression-linear-learner-endpoint
Resource regression-linear-learner-endpoint deleted.


In [23]:
import boto3

# Create a SageMaker client to interact with the SageMaker service
sagemaker_client = boto3.client('sagemaker')

# Define a custom endpoint name for deployment
custom_endpoint_name = "regression-linear-learner-endpoint"

# Deploy the Linear Learner model to the SageMaker endpoint
linear_predictor = linear_learner.deploy(initial_instance_count=1, 
                                         instance_type='ml.m4.xlarge',
                                         endpoint_name=custom_endpoint_name)

INFO:sagemaker:Creating model with name: linear-learner-2024-01-31-14-55-38-473
INFO:sagemaker:Creating endpoint-config with name regression-linear-learner-endpoint
INFO:sagemaker:Creating endpoint with name regression-linear-learner-endpoint


---------!

# Step 8: Query the Endpoint

In [ ]:
from sagemaker.deserializers import JSONDeserializer

# Set the deserializer to JSON
linear_predictor.deserializer = JSONDeserializer()


In [ ]:
# Replace 'your-endpoint-name' with the actual name of your deployed endpoint
response = sagemaker_runtime.invoke_endpoint(EndpointName='your-endpoint-name',
                                             ContentType='text/csv',  # or 'application/json' etc.
                                             Body=input_data_csv)

# Extract and decode the response
result = response['Body'].read().decode('utf-8')
print(result)


In [ ]:
# Sample hardcoded data point
sample_data = [0.00632, 18.0, 2.31, 0, 0.538, 6.575, 65.2, 4.09, 1, 296, 15.3, 396.9, 4.98]

# Convert the sample data to a CSV string
query_data_csv = ','.join([str(item) for item in sample_data])

# Querying the model and getting a prediction
response = linear_predictor.predict(query_data_csv)

# Print out the prediction
print("Predicted value:", response['predictions'][0]['score'])